### 오차역전파법을 적용한 신경망 구현

In [13]:
import sys, os 
sys.path.append(os.pardir)
import numpy as np
from collections import OrderedDict

In [14]:
class ReLU:
    def __init__(self):
        self.mask = None 
        
    def forward(self, x):
        self.mask = (x<=0)
        out = x.copy()
        out[self.mask] = 0 
        return out 
    
    def backward(self, dout):
        dout[self.mask] = 0 
        dx = dout 
        return dx 

In [15]:
def softmax(x):
    c = np.max(x)
    exp_x = np.exp(x-c)
    sum_exp_x = np.sum(exp_x)
    y = exp_x/sum_exp_x
    
    return y

def cross_entropy_error(y, t):
    delta = 1e-7
    return -np.sum(t*np.log(y+delta))


class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None # loss
        self.y = None # output of softmax function
        self.t = None # One-hot vector answer label
        
    def forward(self, x, t):
        self.t = t 
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        return self.loss 
    
    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size 
        
        return dx 

In [16]:
class Affine:
    def __init__(self, w, b):
        self.w = w 
        self.b = b
        self.x = None 
        self.dw = None 
        self.db = None 
        
    def forward(self, x):
        self.x = x 
        out = np.dot(x, self.w) + self.b 
        return out 
    
    def backward(self, dout):
        dx = np.dot(dout, self.w.T) # 여기서 대문자 'T'는 전치행렬을 뜻함, 혹은 np.transpose(w) 또는 np.swapaxes(w,0,1)로 정의 가능 
        self.dw = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)

In [19]:
class TwoLayerNet:
    
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        # Weight reset 
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)
        
        # Create Layers
        self.layers = OrderedDict()
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])
        
        self.lastLayer = SoftmaxWithLoss()
        
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.foward()
            
        return x 
    
    def loss(self, x, t):
        y = self.predict(x)
        return self.lastLayer.forward(y,t)
    
    def accuracy(self,x,t):
        y = self.predict(x)
        y = np.argmasx(y, axis=1)
        if t.ndim !=1 : t = np.argmax(t,axis=1)
        
        accuracy = np.sum(y==t) / float(x.shape[0])
        return accuracy